In [2]:
import torch
import torch.nn as nn
import tiktoken

In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [4]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

In [5]:
# example of unpacking:
primes = [2, 3, 5, 7, 11]
print(*primes)

2 3 5 7 11


In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [8]:
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.1560, -0.2871,  1.4724,  ..., -0.5614,  0.0468,  0.8664],
         [ 1.4226,  0.0499,  0.7210,  ..., -0.5739,  0.8184, -0.4933],
         [-0.2057, -0.1422, -0.6344,  ...,  0.7999,  0.3100, -0.0651],
         [ 0.1019, -0.0208, -0.2520,  ..., -0.8389, -0.4269, -0.2634]],

        [[ 0.0499, -0.2180,  1.5970,  ..., -0.4471,  0.4311,  1.2321],
         [ 1.5826, -1.2463,  2.1012,  ...,  0.8101,  1.5651, -0.4352],
         [ 0.4096,  0.0495, -1.2935,  ...,  0.7633,  0.9806,  0.6068],
         [ 0.2782,  1.0939, -0.6634,  ..., -0.5792, -0.8954,  0.2584]]],
       grad_fn=<UnsafeViewBackward0>)


In [9]:
txt2

'Every day holds a'

In [10]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [11]:
start_context = "Hello, I am"

encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [12]:
encoded

[15496, 11, 314, 716]

In [13]:
torch.tensor(encoded).unsqueeze(0).shape

torch.Size([1, 4])

In [14]:
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 30709, 30379, 15012, 47127,  7662, 32224]])
Output length: 10


In [15]:
out.squeeze(0).tolist()

[15496, 11, 314, 716, 30709, 30379, 15012, 47127, 7662, 32224]

In [16]:
tokenizer.decode( out.squeeze(0).tolist() )

'Hello, I amPART Arlington heritageRum elev Hazard'

In [17]:
tokenizer.decode( [ 37925])

'zos'

In [18]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [19]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

In [20]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves youosityifterPh Question fanc Muk accustomed need – fluffy


In [21]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [22]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [23]:
probas

tensor([[[3.5646e-05, 2.5733e-05, 5.5171e-05,  ..., 1.6939e-05,
          1.7033e-05, 2.5161e-05],
         [3.6211e-05, 1.6699e-05, 3.2103e-05,  ..., 1.0698e-05,
          2.4506e-05, 7.8769e-06],
         [7.2214e-06, 1.2280e-05, 5.6029e-06,  ..., 2.4879e-05,
          1.5543e-05, 1.0807e-05]],

        [[3.2246e-05, 4.0099e-05, 2.4111e-05,  ..., 7.3956e-06,
          9.3326e-06, 2.3436e-05],
         [3.5385e-05, 1.0228e-05, 4.5650e-05,  ..., 4.0222e-06,
          6.6641e-05, 1.9352e-05],
         [5.7606e-06, 2.6992e-05, 1.6573e-05,  ..., 3.0049e-05,
          2.1565e-05, 1.0397e-05]]])

In [24]:
# predicted tokens:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[ 4621],
         [27262],
         [24152]],

        [[34615],
         [34407],
         [18145]]])


In [25]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  advantage Streets Ahmad


In [26]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([2.2509e-05, 2.0195e-05, 2.2738e-05])
Text 2: tensor([2.0900e-05, 2.5703e-05, 4.5818e-06])


In [27]:
# Compute logarithm of all token probabilities
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([-10.7016, -10.8101, -10.6915, -10.7758, -10.5689, -12.2934])


In [28]:
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(-10.9735)
tensor(10.9735)
